In [ ]:
import os
import sys
import os

import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import pyaudio as pa
import struct

In [ ]:

relative_input_path = "input-audios/guitar.wav"
input_path = os.path.abspath(os.path.join(os.getcwd(), relative_input_path))

In [ ]:
y, sr = librosa.load(input_path)

# Compute the Short-Time Fourier Transform (STFT)
hop_length = 512  #adustable
n_fft = 2048  # adustable
stft = librosa.core.stft(y, hop_length=hop_length, n_fft=n_fft)

# Convert amplitude to decibels
stft_db = librosa.amplitude_to_db(np.abs(stft), ref=np.max)

# # Extract time and frequency
time_values_float = librosa.times_like(stft)
frequency_values_float = librosa.fft_frequencies(sr=sr, n_fft=n_fft)

time_values = np.round(time_values_float).astype(int)
frequency_values = np.round(frequency_values_float).astype(int)

min_frequency = 20
max_frequency = 20000

frequency_values = np.clip(frequency_values, min_frequency, max_frequency)

time_values = np.maximum(time_values, 1)
min_length = min(len(time_values), len(frequency_values))
time_values = time_values[:min_length]
frequency_values = frequency_values[:min_length]

print("time_values", time_values)
print("frequency_values", frequency_values)
print("Number of values in time_values:", len(time_values))
print("Number of values in frequency_values:", len(frequency_values))

In [ ]:
import scipy.io.wavfile as wav

def interpolate_linearly(wave_table, index):
    truncated_index = int(np.floor(index))
    next_index = (truncated_index + 1) % wave_table.shape[0]
    next_index_weight = index - truncated_index
    truncated_index_weight = 1 - next_index_weight
    return truncated_index_weight * wave_table[truncated_index] + next_index_weight * wave_table[next_index]


In [ ]:
def fade_in_out(signal, fade_length=1000):
    if fade_length > len(signal):
        fade_length = len(signal)

    fade_in = (1 - np.cos(np.linspace(0, np.pi, fade_length))) * 0.5
    fade_out = np.flip(fade_in)

    signal[:fade_length] = np.multiply(fade_in, signal[:fade_length])

    if len(signal) > fade_length:
        signal[-fade_length:] = np.multiply(fade_out, signal[-fade_length:])

    return signal

In [ ]:
def generate_wave_table(wavetable_length):
    wave_table = np.zeros((wavetable_length,))
    for n in range(wavetable_length):
        wave_table[n] = np.sin(2 * np.pi * n / wavetable_length)
    return wave_table

In [ ]:
all_fequencies_outputs=[]
def wavetable_synthesis(time, frequency, sample_rate):
    wavetable_length = 64
    output = np.zeros((0,))
    wave_table = generate_wave_table(wavetable_length)

    
    for t, f in zip(time, frequency):
        time_value = t
        frequency_value = f
        waveform = np.sin
        
        
        wave_table = np.zeros((wavetable_length,))
        for n in range(wavetable_length):
            wave_table[n] = waveform(2* np.pi * n/ wavetable_length )
            
        output = np.zeros((time_value * sample_rate,))
        index = 0
        indexIncrement = frequency_value * wavetable_length/ sample_rate
        
        for n in range(output.shape[0]):
            output[n] = interpolate_linearly(wave_table, index)
            index += indexIncrement
            index %=wavetable_length
            
        gain = -20
        amplitude = 10 ** (gain /20) 
        output *= amplitude
        output = fade_in_out(output)
        all_fequencies_outputs.append(output)

    return all_fequencies_outputs

def main():
    sample_rate = 44100

    time = time_values
    frequency = frequency_values 
    
    outputs = wavetable_synthesis(time, frequency, sample_rate)

    for i, output in enumerate(outputs):
        print(f"Output for frequency {frequency[i]} Hz: {output}")


if __name__ == "__main__":
    main()
